<a href="https://colab.research.google.com/github/fadelnast/ML-Learning/blob/master/Getting_started_with_Titaninc_dataset.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [0]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))


In [0]:
#Looking into training dataset
dataset_filepath = '../input/titanic/train.csv'
train_dataset = pd.read_csv(dataset_filepath)
train_dataset.head()

In [0]:
#Looking into test dataset
test_dataset = pd.read_csv('../input/titanic/test.csv')
test_dataset.head()


In [0]:
#Checking the probability of women that survive. Base on assumption in gender_submission.csv
women = train_dataset.loc[train_dataset.Sex == 'female']["Survived"]
rate_women = sum(women)/len(women)

print("% of women who survived:", rate_women)

In [0]:
#Checking the probability of men that survive. Base on assumption in gender_submission.csv
men = train_dataset.loc[train_dataset.Sex == 'male']["Survived"]
rate_men = sum(men)/len(men)

print("% of men who survived:", rate_men)

Based on training dataset, it looks like women passenger are more likely to survived in the titanic disaster. That was the basis of gender_submission.csv dataset. It could be improved with several techniques

In [0]:
#Checking the data
train_dataset.describe()

In [0]:
#Checking the missing data
train_dataset.isnull().sum()

In [0]:
#Fill the missing data only in gonna be use features
train_dataset['Age'].fillna(train_dataset['Age'].mean(), inplace=True)
train_dataset.isnull().sum()

In [0]:
#Labels data
Y = train_dataset['Survived']
print(Y.head())
print('\n')

features = ["Pclass", "Sex", "SibSp", "Parch", "Age", "Fare", "Embarked"]
#Features data
X = train_dataset[features]

#scaling the categorical features data
X = pd.get_dummies(train_dataset[features])
print(X.head())
print('\n')


In [0]:
from sklearn.preprocessing import MinMaxScaler

#scalling the numerical features
def ScaleNumerical(data):
    scale=MinMaxScaler()
    data_scaled=scale.fit_transform(data)
    data = pd.DataFrame(data_scaled, columns=data.columns)
    return data.head()

ScaleNumerical(X)

In [0]:
#fill the missing data
test_dataset['Age'].fillna(test_dataset['Age'].mean(), inplace=True)
test_dataset['Fare'].fillna(test_dataset['Fare'].mean(), inplace=True)
test_dataset.isnull().sum()

#Test dataset
X_test = test_dataset[features]
print(X_test)

X_test = pd.get_dummies(test_dataset[features])
print(X_test)
ScaleNumerical(X_test)

In [0]:
from sklearn.model_selection import train_test_split

#Split the training data into train and validaton data
X_train, X_val, Y_train, Y_val = train_test_split(X, Y, random_state=0)

print('X_train shape: ', X_train.shape, '\n')
print('X_val shape: ', X_val.shape, '\n')
print('Y_train shape: ', Y_train.shape, '\n')
print('Y_val shape: ', Y_val.shape, '\n')

In [0]:
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import mean_absolute_error

#building the model
forest_model = RandomForestClassifier(n_estimators=150, max_depth=10, random_state=1)
#fitting the model
forest_model.fit(X_train, Y_train)
#validate the model
val_pred = forest_model.predict(X_val)
print(mean_absolute_error(Y_val, val_pred))

In [0]:
#Predict the model with test data
predictions = forest_model.predict(X_test)
predictions[:10]

In [0]:
output = pd.DataFrame({'PassengerId': test_dataset.PassengerId, 'Survived': predictions})
output.to_csv('my_submission.csv', index=False)
print("Your submission was successfully saved!")